In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from qiskit import QuantumCircuit

qasm_dir = "/Users/debarthipal/Library/CloudStorage/OneDrive-IBM/Desktop/VS files/cutting+obp_new_git_env/QASM_circuits_medium"
qasm_files = sorted([
    f for f in os.listdir(qasm_dir)
    if f.endswith(".qasm")
])
print(qasm_files)

['bigadder_n18.qasm', 'bv_n14.qasm', 'bv_n19.qasm', 'cat_state_n22.qasm', 'cc_n12.qasm', 'dnn_n16.qasm', 'gcm_h6.qasm', 'ghz_state_n23.qasm', 'ising_n26.qasm', 'knn_n25.qasm', 'multiplier_n15.qasm', 'multiply_n13.qasm', 'qec9xz_n17.qasm', 'qf21_n15.qasm', 'qft_n18.qasm', 'sat_n11.qasm', 'seca_n11.qasm', 'square_root_n18.qasm', 'swap_test_n25.qasm', 'wstate_n27.qasm']


In [4]:
from circuit_cut_subexps_copy import perform_cutting_copy
from qiskit_ibm_runtime.fake_provider import FakeTorino
backend = FakeTorino()


In [5]:
from qiskit import QuantumCircuit

def create_copy(circuit : QuantumCircuit)-> QuantumCircuit | None :
    if circuit.num_clbits > 0: # circuit has measurement
        circuit.remove_final_measurements(inplace=True)
    if circuit.num_clbits == 0:
        return circuit
    new_circuit = QuantumCircuit(circuit.num_qubits)
    try:
        for item in circuit.data:
            if item.operation.name == 'measure' or item.operation.name == 'barrier':
                continue

            else:
                if item.operation.num_qubits == 1:
                    q0 = item.qubits[0]._index
                    new_circuit.append(item,q0)
                elif item.operation.num_qubits == 2:
                    q0 = item.qubits[0]._index
                    q1 = item.qubits[1]._index
                    new_circuit.append(item,q0,q1)
        return new_circuit
    except:
        return None

In [6]:
import json
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_addon_obp.utils.simplify import OperatorBudget

def run(qasm_files, max_qwc:int):
    circ_dic={}
    pm = generate_preset_pass_manager(optimization_level=3, basis_gates=backend.configuration().basis_gates, seed_transpiler=1)

    for index, file in enumerate(qasm_files, start=1):
        n=QuantumCircuit.from_qasm_file(file).num_qubits
        observable_terms = [
                "I"*(i-1) + "Z" + "I"*(n - i)
                for i in range(1, n+1)
            ]
        observable = SparsePauliOp(observable_terms, coeffs=[1/(n)] * (n))
        circuit=QuantumCircuit.from_qasm_file(file)
        num_qubits_circuit = circuit.num_qubits
        circuit = pm.run(circuit)
        synth_circuit_depth = circuit.depth()
        circuit = create_copy(circuit)
        if type(circuit) is not QuantumCircuit:
            circ_dic[index] = {"filename":file, 
                               "num_qubits":num_qubits_circuit,
                               "synth_ckt_depth":synth_circuit_depth,
                               "number_of_cuts":number_of_cuts,
                               "number_of_obp_cuts":number_of_obp_cuts,
                                "num_subexp_without_obp": None, 
                                "num_subexp_with_obp": None
                                }
        else:
            op_budget = OperatorBudget(max_qwc_groups=max_qwc)
            number_of_cuts,number_of_obp_cuts,number_subexp_without_obp,number_subexp_with_obp = perform_cutting_copy(circuit, 
                                                                                                          observable,backend,op_budget)
            circ_dic[index] = {"filename":file, 
                               "num_qubits":num_qubits_circuit,
                               "synth_ckt_depth":synth_circuit_depth,
                               "number_of_cuts":number_of_cuts,
                               "number_of_obp_cuts":number_of_obp_cuts,
                                "num_subexp_without_obp": number_subexp_without_obp, 
                                "num_subexp_with_obp":number_subexp_with_obp,
                                }
            
        json_obj=json.dumps(circ_dic, indent=4)
        file_name = "medium_ckt_files_with_max_qwc_" + str(max_qwc) + ".json"
        with open(file_name, "w") as f:
            f.write(json_obj)   #Saving to a JSON file

In [7]:
#test_qasm_file = [qasm_files[2]]
for max_qwc_group in range(1,6):
    print(f"Starting for {max_qwc_group}")
    run(qasm_files,max_qwc_group)

Starting for 1
Starting for 2
Starting for 3
Starting for 4


Reached the specified max_seconds timeout!


Starting for 5


Reached the specified max_seconds timeout!
